# Extract, transform, and load data

All data has been scraped at this point. This notebook focuses on ETL.

In [1]:
import pandas as pd
import numpy as np
import bs4
import requests
import neweggutils
import re

from collections import OrderedDict

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('max_colwidth', 200)

In [3]:
def extract_load(components, prices):
    '''
        Parses HTML of two dataframes and merges them in to a single dataframe.
        ====Parameters====
        components: html of individual computer pages
        prices: html of product array page with 96 products
        ====Returns====
        merged: dataframe with prices corresponding to components
    '''
    try:
        prices.drop(labels=['Unnamed: 0', 'component_html'], axis=1, inplace=True)
        components.drop(labels=['Unnamed: 0', 'price_html'], axis=1, inplace=True)

    except (KeyError, ValueError):
        pass
    
    components = components.apply(lambda x: neweggutils.get_components(x[0]), axis=1)
    prices = prices.apply(lambda x: neweggutils.get_prices_and_links(x[0]))# axis=1) if prices is df
    
    components = components.dropna()
    prices = prices.dropna()
    
    components = pd.DataFrame.from_records(components.values)    
    
    # Concatenate all rows in to one list of prices and links
    concat_prices = []
    
    for price in prices:
        concat_prices.extend(price)
                
    prices = pd.DataFrame(concat_prices)
    prices.columns = ['price', 'link']
    
    merged = prices.merge(components, on='link', how='left')
    
    return merged

## Get all available data in to one dataframe

First load the available HTML dataframes.

In [4]:
f5c = pd.read_csv('first_5_prices_backup.csv', index_col=0)
f5p = pd.read_csv('first_5_components_backup.csv', index_col=0)

In [5]:
f5c.reset_index(inplace=True, drop=True)

In [6]:
f5p.reset_index(inplace=True, drop=True)

In [7]:
s5p = pd.read_csv('6_to_11_price.csv', index_col=0)
s5c = pd.read_csv('6_to_11_component.csv', index_col=0)

In [8]:
t5c = pd.read_csv('16_price.csv', index_col=0)
t5p = pd.read_csv('16_comp.csv', index_col=0)

In [12]:
t5c.reset_index(inplace=True, drop=True)
t5p.reset_index(inplace=True, drop=True)

In [11]:
last_c = pd.read_csv('32-on-comp.csv', index_col=0)
last_p = pd.read_csv('32-on-prices.csv', index_col=0)

last_c.reset_index(inplace=True, drop=True)
last_p.reset_index(inplace=True, drop=True)

In [13]:
df1 = neweggutils.extract_load(f5c, f5p)

In [14]:
df1.to_csv('df1_final.csv')

In [15]:
df2 = neweggutils.extract_load(s5c, s5p)

In [16]:
df2.to_csv('df2_final.csv')

In [17]:
df3 = neweggutils.extract_load(t5c, t5p)

In [18]:
df3.to_csv('df3_final.csv')

In [19]:
df4 = neweggutils.extract_load(last_c, last_p)

In [20]:
df4.to_csv('df4_final.csv')

In [24]:
final = pd.concat([df1, df2, df3, df4], sort=False)

In [25]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4628 entries, 0 to 1275
Data columns (total 92 columns):
price                             4626 non-null object
link                              4628 non-null object
Brand                             3174 non-null object
Series                            2781 non-null object
Model                             4285 non-null object
Part Number                       1536 non-null object
Type                              1285 non-null object
Form Factor                       3985 non-null object
Usage                             1152 non-null object
Colors                            3557 non-null object
Processor                         2965 non-null object
Processor Main Features           1281 non-null object
Cache Per Processor               1021 non-null object
Memory                            2851 non-null object
Storage                           2821 non-null object
Optical Drive                     2498 non-null object
Graphics     

In [26]:
final.to_csv('final.csv')

In [28]:
final.duplicated().mean()

0.35738980121002595